In [ ]:
import tensorflow as tf
import findspark

In [ ]:
print(tf.__version__)

In [ ]:
print(findspark.find())
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("CNN") \
                    .getOrCreate()

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print('Image Inventory')
print('----------')
print('Training: {}'.format(len(x_train)))
print('Testing:  {}'.format(len(x_test)))
print('----------')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
for i in range(2):
    image = x_train[i]
    image = np.array(image, dtype='float')
    label = y_train[i]
    pixels = image.reshape((28, 28))
    plt.imshow(pixels, cmap='gray')
    print('-----------------')
    print(label)
    plt.show()
    

In [ ]:
from matplotlib import image
for i in range(1,10):
    png = x_train[i]
    png = np.array(png, dtype='float')
    pixels = png.reshape((28, 28))
    image.imsave('image_no_{}.png'.format(i), pixels, cmap = 'gray')

In [ ]:
import os
print(os.listdir())

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [ ]:
x_train = x_train.reshape(60000, 28,28,1)
x_test = x_test.reshape(10000, 28,28,1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
print(y_train[0].shape)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,batch_size=512,
          epochs=5,
          validation_data=(x_test, y_test))

In [ ]:
stats = model.evaluate(x_test, y_test)
print('The accuracy rate is {}%'.format(round(stats[1],3)*100))
print('The loss rate is {}%'.format(round(stats[0],2)*100))

In [ ]:
model.summary()

In [ ]:
filters, biases = model.layers[0].get_weights()

f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
print(filters.shape)

In [ ]:
plt.figure(figsize=(16, 16)) 
ix = 1
for i in range(32):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    
    # specify subplot and turn of axis
    ax = plt.subplot(8, 4, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    plt.imshow(f[:, :], cmap='gray')
    ix = ix + 1
# show the figure
plt.show()

In [ ]:
feature_model = Model(inputs=model.inputs, outputs=model.layers[1].output)

In [ ]:
feature_maps = feature_model.predict(x_test[125:126])
print((feature_maps.shape))

In [ ]:
plt.figure(figsize=(16, 16)) 
ix = 1
for i in range(32):
     ax = plt.subplot(8, 4, ix)
	#ax.set_xticks([])
	#ax.set_yticks([])
     plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
     ix += 1
plt.show()

In [ ]:
y_test[125:126]

In [ ]:
spark.stop()